## TensorFlow Core 
Used in machine learning research for better control over parameters

In [38]:
import tensorflow as tf

#create floating point tensors 
#nodes define computational graphs, they take tensors as inputs and produce
#tensors as outputs 
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0) #also tf.float32 implicitly
print (node1, node2)

Tensor("Const_6:0", shape=(), dtype=float32) Tensor("Const_7:0", shape=(), dtype=float32)


In [39]:
#sessions encapsulate the control and state of the tensorflow runtime 
sess = tf.Session()
print (sess.run([node1, node2]))
#this would evaluate the nodes 

[3.0, 4.0]


In [40]:
from __future__ import print_function 
node3 = tf.add(node1,node2) #operations are also nodes 
print ("node3: ", node3)
print ("sess.run(node3): ", sess.run(node3))

node3:  Tensor("Add_3:0", shape=(), dtype=float32)
sess.run(node3):  7.0


In [41]:
#parametrized nodes have placeholders for future values 
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b #shortcut version of tf.add(a,b), adding must auto invoke

print(sess.run(adder_node, {a:3, b:4.5}))
print(sess.run(adder_node, {a:[1,3], b:[2,4]}))

7.5
[ 3.  7.]


In [42]:
add_and_triple = adder_node * 3
print (sess.run(add_and_triple, {a:3, b:4.5} ))

22.5


In [43]:
#variables are trainable parameters 
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(tf.float32) 
linear_model = W*x + b

#unlike constants, variables are not initalized untill you call
init = tf.global_variables_initializer()
sess.run(init) #passing init subhandle to the running of our session inits 


In [44]:
# run linear model by passing values of x to it 
print (sess.run(linear_model, {x:[1,2,3,4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


In [45]:
#performing loss analysis 
y=tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model-y) #squared difference 
loss = tf.reduce_sum(squared_deltas)
print (sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]}))

23.66


In [46]:
fixW = tf.assign(W,[-1.])
fixb = tf.assign(b,[1.]) #this is how to reassign initialized vars 
sess.run([fixW,fixb])
print (sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]}))

0.0


Training in tensorflow

In [47]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

sess.run(init) # reset vals to incorrect defaults 
for i in range(1000):
    sess.run(train, {x:[1,2,3,4], y:[0,-1,-2,-3]})
                
print(sess.run([W,b]))

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


In [48]:
#All code put together using tensorflow core 
import tensorflow as tf

# Model parameters
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
# Model input and output
x = tf.placeholder(tf.float32)
linear_model = W*x + b
y = tf.placeholder(tf.float32)

# loss
loss = tf.reduce_sum(tf.square(linear_model - y)) # sum of the squares
# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

# training data
x_train = [1, 2, 3, 4]
y_train = [0, -1, -2, -3]
# training loop
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init) # reset values to wrong
for i in range(1000):
  sess.run(train, {x: x_train, y: y_train})

# evaluate training accuracy
curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x: x_train, y: y_train})
print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

W: [-0.9999969] b: [ 0.99999082] loss: 5.69997e-11


## TensorFlow Estimator for fast implementation 

In [1]:
import numpy as np
import tensorflow as tf 

#declare list of features, we have one numeric features 
feature_columns = [tf.feature_column.numeric_column("x", shape=[1])]

#we choose the linear regression estimator 
estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns)

#define our data
x_train = np.array([1.,2.,3.,4.])
y_train = np.array([0,-1.,-2.,-3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])

#create helper methods 
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x":x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

#train for 1000 steps
estimator.train(input_fn=input_fn, steps=1000)

#eval
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': 'worker', '_keep_checkpoint_every_n_hours': 10000, '_num_worker_replicas': 1, '_save_checkpoints_steps': None, '_model_dir': 'C:\\Users\\Raed\\AppData\\Local\\Temp\\tmp3yuk6448', '_master': '', '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_is_chief': True, '_task_id': 0, '_log_step_count_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001C034D812E8>, '_session_config': None, '_service': None, '_tf_random_seed': None, '_save_summary_steps': 100, '_num_ps_replicas': 0}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\Raed\AppData\Local\Temp\tmp3yuk6448\model.ckpt.
INFO:tensorflow:step = 1, loss = 5.0
INFO:tensorflow:global_step/sec: 919.223
INFO:tensorflow:step = 101, loss = 0.0583803 (0.110 sec)
INFO:tensorflow:global_step/sec: 1303.71
INFO:tensorflow:step = 201, loss = 0.00358104 (0.077 sec)
IN

In [4]:
import numpy as np
import tensorflow as tf

# Declare list of features, we only have one real-valued feature
def model_fn(features, labels, mode):
  # Build a linear model and predict values
  W = tf.get_variable("W", [1], dtype=tf.float64)
  b = tf.get_variable("b", [1], dtype=tf.float64)
  y = W*features['x'] + b
  # Loss sub-graph
  loss = tf.reduce_sum(tf.square(y - labels))
  # Training sub-graph
  global_step = tf.train.get_global_step()
  optimizer = tf.train.GradientDescentOptimizer(0.01)
  train = tf.group(optimizer.minimize(loss),
                   tf.assign_add(global_step, 1))
  # EstimatorSpec connects subgraphs we built to the
  # appropriate functionality.
  return tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=y,
      loss=loss,
      train_op=train)

estimator = tf.estimator.Estimator(model_fn=model_fn)
# define our data sets
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7., 0.])
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

# train
estimator.train(input_fn=input_fn, steps=1000)
# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': 'worker', '_keep_checkpoint_every_n_hours': 10000, '_num_worker_replicas': 1, '_save_checkpoints_steps': None, '_model_dir': 'C:\\Users\\Raed\\AppData\\Local\\Temp\\tmp2euzn8ih', '_master': '', '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_is_chief': True, '_task_id': 0, '_log_step_count_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001C04029EEF0>, '_session_config': None, '_service': None, '_tf_random_seed': None, '_save_summary_steps': 100, '_num_ps_replicas': 0}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\Raed\AppData\Local\Temp\tmp2euzn8ih\model.ckpt.
INFO:tensorflow:step = 1, loss = 51.4128225996
INFO:tensorflow:global_step/sec: 1375.65
INFO:tensorflow:step = 101, loss = 0.000201647533238 (0.074 sec)
INFO:tensorflow:global_step/sec: 1662.24
INFO:tensorflow:step = 201, loss = 2.92846